<a href="https://colab.research.google.com/github/sahil301290/TSS2022-ThaparSummerSchool/blob/main/29Jun2022_Face_Age_Gender_Expression_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DeepFace - Age, Gender, Expression, Headpose and Recognition**

---


In this lesson, we use the **DeepFace API for Age, Gender, Expression Facial and Recognition. We even use the headpose library to obtain head direction/tilt**. DeepFace is an easy to use python module that provides access to several Facial Detection and Recognition models. It's very simple to use to let's dive in.

1. Install the necessary modules and download our files
2. Demonstrate facial landmarks
3. Obtain Age, Gender, Emotional Expression and Ethnicity using DeepFace
4. Perform Facial Similarity
5. Perform Facial Recognition


**NOTE** Change to High-RAM setting.

## **1. Install the necessary modules and download our files**

In [ ]:
!pip install deepface
!pip install dlib

#### **Define our imshow function**

In [ ]:
# Some imports and our image viewing function
import dlib
import tarfile
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Define our imshow function 
def imshow(title = "Image", image = None, size = 6):
    w, h = image.shape[0], image.shape[1]
    aspect_ratio = w/h
    plt.figure(figsize=(size * aspect_ratio,size))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.show()

In [ ]:
# Download facial landmarks
!wget https://moderncomputervision.s3.eu-west-2.amazonaws.com/shape_predictor_68_face_landmarks.dat

####Getting Test Images and Test Pic

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!unzip '/content/drive/MyDrive/TSS2022_Datasets/face_recognition.zip'

## **2. Demonstrate facial landmarks**

In [ ]:
from imutils import face_utils

p = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

image = cv2.imread('sahil1.JPG')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
# Get faces 
rects = detector(gray, 0)

# For each detected face, find the landmark.
for (i, rect) in enumerate(rects):
    # Make the prediction and transfom it to numpy array
    shape = predictor(gray, rect)
    shape = face_utils.shape_to_np(shape)
    (x, y, w, h) = face_utils.rect_to_bb(rect)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    # Draw on our image, all the finded cordinate points (x,y) 
    for (x, y) in shape:
        cv2.circle(image, (x, y), 2, (0, 255, 0), -1)

# Show the image
imshow("Output", image)

## **3. Obtain Age, Gender, Emotional Expression and Ethnicity using DeepFace**

**Download our models**

In [ ]:
!gdown --id 1Id32-d-nS9BooBLLkw1PQhvLWWAukCsq
!gdown --id 1txWignSWdELl8cWdZHYqIlSE2ZRjI8WI
!gdown --id 1d_tQRWjvQ5i4lZyUfFEfRj7LzXWXseBY
!gdown --id 1kWp2CVg_xTIFqdZAwfN_86A3grim9NyI

!mv facial_expression_model_weights.zip /root/.deepface/weights/facial_expression_model_weights.zip
!mv age_model_weights.h5 /root/.deepface/weights/age_model_weights.h5
!mv gender_model_weights.h5 /root/.deepface/weights/gender_model_weights.h5
!mv race_model_single_batch.zip /root/.deepface/weights/race_model_single_batch.zip

In [ ]:
from deepface import DeepFace

In [ ]:
obj = DeepFace.analyze(img_path =  "./face_recognition/people/sahil.jpg", actions = ['age', 'gender', 'race', 'emotion'])
print(obj["age"]," years old ",obj["dominant_race"]," ",obj["dominant_emotion"]," ", obj["gender"])

In [ ]:
from deepface import DeepFace
import pprint

img_path = "./face_recognition/people/sahil.jpg"
image = cv2.imread(img_path)

obj = DeepFace.analyze(img_path = img_path,
                       actions = ['age', 'gender', 'race', 'emotion'])
imshow("Face Analysis", image)
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(obj)

#### **Create a simple function to display our results on the image**

In [ ]:
import cv2

def drawFace(img_path, obj):
  image = cv2.imread(img_path)
  x = obj['region']['x'] 
  y = obj['region']['y'] 
  h = obj['region']['h'] 
  w = obj['region']['w'] 
  age = obj['age']
  gender = obj['gender']
  gender = 'F' if gender == 'Woman' else 'M'
  dominant_emotion = obj['dominant_emotion']
  dominant_race = obj['dominant_race']
  dominant_emotion = obj['dominant_emotion']
  description = f'{age}{gender} - {dominant_emotion}'
  cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
  cv2.putText(image, description, (x,y-10) , cv2.FONT_HERSHEY_PLAIN,2, (0,255,0), 3)
  cv2.putText(image, dominant_race, (x,y+h+30) , cv2.FONT_HERSHEY_PLAIN,2, (0,255,0), 3)
  imshow("Face Analysis", image)

#### **Test on another image**

In [ ]:
from deepface import DeepFace
import pprint

img_path = "/content/sahil1.JPG"
image = cv2.imread(img_path)
obj = DeepFace.analyze(img_path = img_path, enforce_detection=False, actions = ['age', 'gender', 'race', 'emotion'])
drawFace(img_path, obj)
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(obj)

#### **Change backends of face detection**

In [ ]:
from deepface import DeepFace

# backends = ['opencv', 'ssd', 'dlib', 'mtcnn', 'retinaface']

img_path = "./sahil1.JPG"
image = cv2.imread(img_path)
obj = DeepFace.analyze(img_path = "./sahil1.JPG", actions = ['age', 'gender', 'race', 'emotion'], detector_backend = 'ssd')
drawFace(img_path, obj)
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(obj)

## **4. Perform Facial Similarity**

In [ ]:
result  = DeepFace.verify("./face_recognition/people/sahil.jpg", "./sahil1.JPG", enforce_detection=False)
print("Is verified: ", result["verified"])
result

### **We can even use different Distance Metrics**

In [ ]:
#metrics = ["cosine", "euclidean", "euclidean_l2"]

result  = DeepFace.verify("./face_recognition/people/sahil.jpg", "./sahil1.JPG", enforce_detection=False, distance_metric = 'euclidean')
print("Is verified: ", result["verified"])
result

In [ ]:
#metrics = ["cosine", "euclidean", "euclidean_l2"]

result  = DeepFace.verify("./face_recognition/people/sahil.jpg", "./sahil1.JPG", enforce_detection=False, distance_metric = 'euclidean_l2')
print("Is verified: ", result["verified"])
result

### **Download models as the existing DeepFace downloader has stopped working**

In [ ]:
!gdown --id 1OdJNKL85CCYStVi9XtJRpHhXo2FU6Gf1
!gdown --id 1GWIuvW3Vm3wMpGGEyTT7sU-c1cVWZIEc
!mv vgg_face_weights.h5 /root/.deepface/weights/vgg_face_weights.h5
!mv facenet_weights.h5 /root/.deepface/weights/facenet_weights.h5

## **5. Perform Facial Recognition**

In [ ]:
from deepface import DeepFace
import pandas as pd

df = DeepFace.find(img_path = "./face_recognition/training_faces/Nidia_1.jpg", db_path = './face_recognition/training_faces/', detector_backend = 'ssd')
df

## **We can even try a few different models**

In [ ]:
from deepface import DeepFace
import pandas as pd

dfs = []
models = ["VGG-Face", "Facenet", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib"]

for model in models:
   df = DeepFace.find(img_path = "./face_recognition/training_faces/Nidia_1.jpg", db_path = './face_recognition/training_faces/', model_name = model,  detector_backend = 'ssd')
   df['model'] = model
   dfs.append(df)

pd.concat(dfs)

In [ ]:
imshow('1', cv2.imread('./face_recognition/training_faces/Nidia_1.jpg'))
imshow('1', cv2.imread('./face_recognition/training_faces/Nidia_5.jpg'))

End of Code